In [1]:
import os

root = "ee782_re_project"
subdirs = [
    "data/REFinD",
    "data/FIRE",
    "data/FinRED",
    "src/datasets",
    "src/models",
    "src/training",
    "src/analysis",
    "outputs/checkpoints",
    "outputs/logs",
    "outputs/predictions",
]

for d in subdirs:
    os.makedirs(os.path.join(root, d), exist_ok=True)

root


'ee782_re_project'

In [3]:
import os, zipfile

ROOT = "ee782_re_project"
refind_dir = os.path.join(ROOT, "data", "REFinD")

print("Files currently in REFinD dir:", os.listdir(refind_dir))

# 1) Unzip every .zip into a folder with the same base name
for fname in os.listdir(refind_dir):
    if fname.lower().endswith(".zip"):
        zip_path = os.path.join(refind_dir, fname)
        out_dir = os.path.join(refind_dir, os.path.splitext(fname)[0])
        print(f"Extracting {fname} -> {out_dir}")
        os.makedirs(out_dir, exist_ok=True)
        with zipfile.ZipFile(zip_path, "r") as zf:
            zf.extractall(out_dir)

print("\nAfter extraction:")
for root, dirs, files in os.walk(refind_dir):
    rel = os.path.relpath(root, refind_dir)
    print(f"[{rel}]")
    for f in files:
        print("  ", f)


Files currently in REFinD dir: ['public_dat.zip', 'sample.zip']
Extracting public_dat.zip -> ee782_re_project\data\REFinD\public_dat
Extracting sample.zip -> ee782_re_project\data\REFinD\sample

After extraction:
[.]
   public_dat.zip
   sample.zip
[public_dat]
   dev_refind_official.json
   test_refind_official.json
   train_refind_official.json
[sample]
   sample.csv


In [5]:
import os, json, itertools

ROOT = "ee782_re_project"
refind_dir = os.path.join(ROOT, "data", "REFinD")

print("=== Directory tree under REFinD ===")
for root, dirs, files in os.walk(refind_dir):
    rel = os.path.relpath(root, refind_dir)
    print(f"[{rel}]")
    for f in files:
        print("   ", f)

# Try to automatically find a main data file (json/jsonl/csv)
cand_files = []
for root, dirs, files in os.walk(refind_dir):
    for f in files:
        if f.lower().endswith((".json", ".jsonl", ".csv")):
            cand_files.append(os.path.join(root, f))

print("\n=== Candidate data files ===")
for i, path in enumerate(cand_files):
    print(f"{i}: {path}")


=== Directory tree under REFinD ===
[.]
    public_dat.zip
    sample.zip
[public_dat]
    dev_refind_official.json
    test_refind_official.json
    train_refind_official.json
[sample]
    sample.csv

=== Candidate data files ===
0: ee782_re_project\data\REFinD\public_dat\dev_refind_official.json
1: ee782_re_project\data\REFinD\public_dat\test_refind_official.json
2: ee782_re_project\data\REFinD\public_dat\train_refind_official.json
3: ee782_re_project\data\REFinD\sample\sample.csv


In [7]:
import json, itertools

i = 2  # index for train_refind_official.json from your candidate list
sample_path = cand_files[i]
print("Using sample_path =", sample_path)

with open(sample_path, "r", encoding="utf-8") as f:
    for line in itertools.islice(f, 3):
        ex = json.loads(line)
        print(json.dumps(ex, indent=2)[:1000])
        print("-" * 80)


Using sample_path = ee782_re_project\data\REFinD\public_dat\train_refind_official.json
[
  {
    "id": null,
    "docid": "2016/2017",
    "relation": "no_relation",
    "rel_group": "ORG-ORG",
    "token": [
      "warrants",
      "to",
      "purchase",
      "Lumos",
      "Networks",
      "Corp.",
      "common",
      "stock",
      ",",
      "the",
      "Pamplona",
      "Entities",
      ")",
      "."
    ],
    "e1_start": 3,
    "e1_end": 6,
    "e2_start": 9,
    "e2_end": 12,
    "e1_type": "ORG",
    "e2_type": "ORG",
    "spacy_pos": [
      "NNS",
      "TO",
      "VB",
      "NNP",
      "NNP",
      "NNP",
      "JJ",
      "NN",
      ",",
      "DT",
      "NNP",
      "NNPS",
      "-RRB-",
      "."
    ],
    "spacy_ner": [
      "O",
      "O",
      "O",
      "ORG",
      "ORG",
      "ORG",
      "O",
      "O",
      "O",
      "ORG",
      "ORG",
      "ORG",
      "O",
      "O"
    ],
    "spacy_head": [
      0,
      2,
      0,
      5,
      5,
  

In [3]:
import os
from collections import Counter
import json
ROOT = "ee782_re_project"
refind_pub = os.path.join(ROOT, "data", "REFinD", "public_dat")

def load_refind_split(split_name):
    """
    split_name: 'train', 'dev', or 'test'
    Loads the *_refind_official.json file, which is a JSON array.
    """
    fname = f"{split_name}_refind_official.json"
    path = os.path.join(refind_pub, fname)
    print(f"Loading {split_name} from {path}")
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)  # list of dicts
    print(f"  -> {len(data)} examples")
    return data

train_raw = load_refind_split("train")
dev_raw   = load_refind_split("dev")
test_raw  = load_refind_split("test")

# sanity check one example
print("\nExample from train:")
print(json.dumps(train_raw[0], indent=2)[:1000])


Loading train from ee782_re_project\data\REFinD\public_dat\train_refind_official.json
  -> 20070 examples
Loading dev from ee782_re_project\data\REFinD\public_dat\dev_refind_official.json
  -> 4306 examples
Loading test from ee782_re_project\data\REFinD\public_dat\test_refind_official.json
  -> 4300 examples

Example from train:
{
  "id": null,
  "docid": "2016/2017",
  "relation": "no_relation",
  "rel_group": "ORG-ORG",
  "token": [
    "warrants",
    "to",
    "purchase",
    "Lumos",
    "Networks",
    "Corp.",
    "common",
    "stock",
    ",",
    "the",
    "Pamplona",
    "Entities",
    ")",
    "."
  ],
  "e1_start": 3,
  "e1_end": 6,
  "e2_start": 9,
  "e2_end": 12,
  "e1_type": "ORG",
  "e2_type": "ORG",
  "spacy_pos": [
    "NNS",
    "TO",
    "VB",
    "NNP",
    "NNP",
    "NNP",
    "JJ",
    "NN",
    ",",
    "DT",
    "NNP",
    "NNPS",
    "-RRB-",
    "."
  ],
  "spacy_ner": [
    "O",
    "O",
    "O",
    "ORG",
    "ORG",
    "ORG",
    "O",
    "O",
    "O"

In [9]:
from itertools import chain

all_rels = sorted(
    {ex["relation"] for ex in chain(train_raw, dev_raw, test_raw)}
)
print("Number of unique relations:", len(all_rels))
print(all_rels)

rel2id = {rel: i for i, rel in enumerate(all_rels)}
id2rel = {i: rel for rel, i in rel2id.items()}

print("\nLabel mapping (first few):")
for i in range(min(10, len(all_rels))):
    print(i, "->", id2rel[i])


Number of unique relations: 22
['no_relation', 'org:date:acquired_on', 'org:date:formed_on', 'org:gpe:formed_in', 'org:gpe:headquartered_in', 'org:gpe:operations_in', 'org:money:cost_of', 'org:money:loss_of', 'org:money:profit_of', 'org:money:revenue_of', 'org:org:acquired_by', 'org:org:agreement_with', 'org:org:shares_of', 'org:org:subsidiary_of', 'pers:gov_agy:member_of', 'pers:org:employee_of', 'pers:org:founder_of', 'pers:org:member_of', 'pers:title:title', 'pers:univ:attended', 'pers:univ:employee_of', 'pers:univ:member_of']

Label mapping (first few):
0 -> no_relation
1 -> org:date:acquired_on
2 -> org:date:formed_on
3 -> org:gpe:formed_in
4 -> org:gpe:headquartered_in
5 -> org:gpe:operations_in
6 -> org:money:cost_of
7 -> org:money:loss_of
8 -> org:money:profit_of
9 -> org:money:revenue_of


In [13]:
rel_counts = Counter(ex["relation"] for ex in train_raw)
print("\nMost common relations in train:")
for rel, c in rel_counts.most_common(15):
    print(f"{rel:20s} : {c}")



Most common relations in train:
no_relation          : 9128
pers:title:title     : 3126
org:gpe:operations_in : 2832
pers:org:employee_of : 1733
org:org:agreement_with : 653
org:date:formed_on   : 448
pers:org:member_of   : 441
org:org:subsidiary_of : 386
org:org:shares_of    : 286
org:money:revenue_of : 217
org:money:loss_of    : 141
org:gpe:headquartered_in : 135
org:date:acquired_on : 134
pers:org:founder_of  : 92
org:gpe:formed_in    : 81


In [7]:
def normalize_example(ex):
    """
    Turn one raw REFinD dict into a friendlier format for our Dataset.
    """
    tokens = ex["token"]
    e1 = {
        "start": ex["e1_start"],
        "end":   ex["e1_end"],   # NOTE: end is exclusive
        "type":  ex["e1_type"],
    }
    e2 = {
        "start": ex["e2_start"],
        "end":   ex["e2_end"],   # exclusive
        "type":  ex["e2_type"],
    }
    rel_label = ex["relation"]
    return {
        "docid":   ex["docid"],
        "tokens":  tokens,
        "e1":      e1,
        "e2":      e2,
        "rel":     rel_label,
        "rel_id":  rel2id[rel_label],
        "rel_group": ex["rel_group"],
    }

train_proc = [normalize_example(ex) for ex in train_raw]
dev_proc   = [normalize_example(ex) for ex in dev_raw]
test_proc  = [normalize_example(ex) for ex in test_raw]

print("Train/dev/test sizes (processed):", len(train_proc), len(dev_proc), len(test_proc))
print("\nProcessed example:")
print(train_proc[0])


Train/dev/test sizes (processed): 20070 4306 4300

Processed example:
{'docid': '2016/2017', 'tokens': ['warrants', 'to', 'purchase', 'Lumos', 'Networks', 'Corp.', 'common', 'stock', ',', 'the', 'Pamplona', 'Entities', ')', '.'], 'e1': {'start': 3, 'end': 6, 'type': 'ORG'}, 'e2': {'start': 9, 'end': 12, 'type': 'ORG'}, 'rel': 'no_relation', 'rel_id': 0, 'rel_group': 'ORG-ORG'}


In [23]:
!pip install transformers sentencepiece numpy scipy pyyaml tqdm scikit-learn --quiet



In [11]:
from transformers import AutoTokenizer

# 👇 change this later to "SALT-NLP/FLANG-BERT" for FLANG experiments
MODEL_NAME = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer


BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [9]:
import torch
from torch.utils.data import Dataset

class RefindDataset(Dataset):
    def __init__(self, examples, tokenizer, max_len=256):
        """
        examples: list of processed dicts from train_proc/dev_proc/test_proc
        tokenizer: a HuggingFace tokenizer
        """
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        ex = self.examples[idx]
        tokens = ex["tokens"]
        e1 = ex["e1"]
        e2 = ex["e2"]
        label_id = ex["rel_id"]

        # Tokenize pre-split words
        encoding = self.tokenizer(
            tokens,
            is_split_into_words=True,
            truncation=True,
            max_length=self.max_len,
            return_attention_mask=True,
            return_token_type_ids=True,
        )

        # word_ids maps each subword to its original word index (or None for special tokens)
        word_ids = encoding.word_ids()

        # Build masks for entity 1 and entity 2 over subword positions
        subj_mask = [0] * len(word_ids)
        obj_mask  = [0] * len(word_ids)

        for i, w_id in enumerate(word_ids):
            if w_id is None:
                continue
            if e1["start"] <= w_id < e1["end"]:
                subj_mask[i] = 1
            if e2["start"] <= w_id < e2["end"]:
                obj_mask[i] = 1

        # Convert everything to tensors (leave padding to collate_fn)
        item = {
            "input_ids": torch.tensor(encoding["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(encoding["attention_mask"], dtype=torch.long),
            "subj_mask": torch.tensor(subj_mask, dtype=torch.float),
            "obj_mask": torch.tensor(obj_mask, dtype=torch.float),
            "labels": torch.tensor(label_id, dtype=torch.long),
        }
        # token_type_ids may not exist for some models (e.g., RoBERTa); handle safely
        if "token_type_ids" in encoding:
            item["token_type_ids"] = torch.tensor(encoding["token_type_ids"], dtype=torch.long)

        return item


In [11]:
# Create Dataset objects for each split
train_ds = RefindDataset(train_proc, tokenizer, max_len=256)
dev_ds   = RefindDataset(dev_proc,   tokenizer, max_len=256)
test_ds  = RefindDataset(test_proc,  tokenizer, max_len=256)

print("Dataset sizes:", len(train_ds), len(dev_ds), len(test_ds))
print("Example item keys:", train_ds[0].keys())


NameError: name 'train_proc' is not defined

In [27]:
from torch.utils.data import DataLoader
import torch

def collate_fn(batch):
    # batch is a list of dicts from RefindDataset.__getitem__
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0

    # find max sequence length in this batch
    max_len = max(len(ex["input_ids"]) for ex in batch)

    def pad_seq(seq, pad_value):
        seq = seq.tolist() if isinstance(seq, torch.Tensor) else list(seq)
        return seq + [pad_value] * (max_len - len(seq))

    input_ids = torch.tensor(
        [pad_seq(ex["input_ids"], pad_token_id) for ex in batch],
        dtype=torch.long,
    )
    attention_mask = torch.tensor(
        [pad_seq(ex["attention_mask"], 0) for ex in batch],
        dtype=torch.long,
    )
    subj_mask = torch.tensor(
        [pad_seq(ex["subj_mask"], 0.0) for ex in batch],
        dtype=torch.float,
    )
    obj_mask = torch.tensor(
        [pad_seq(ex["obj_mask"], 0.0) for ex in batch],
        dtype=torch.float,
    )
    labels = torch.tensor(
        [int(ex["labels"]) for ex in batch],
        dtype=torch.long,
    )

    batch_dict = {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "subj_mask": subj_mask,
        "obj_mask": obj_mask,
        "labels": labels,
    }

    # handle token_type_ids if present
    if "token_type_ids" in batch[0]:
        token_type_ids = torch.tensor(
            [pad_seq(ex["token_type_ids"], 0) for ex in batch],
            dtype=torch.long,
        )
        batch_dict["token_type_ids"] = token_type_ids

    return batch_dict

# Rebuild DataLoaders with the new collate_fn
BATCH_SIZE = 16
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
dev_loader   = DataLoader(dev_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# Sanity check one batch
batch = next(iter(train_loader))
for k, v in batch.items():
    print(k, v.shape)


input_ids torch.Size([16, 100])
attention_mask torch.Size([16, 100])
subj_mask torch.Size([16, 100])
obj_mask torch.Size([16, 100])
labels torch.Size([16])
token_type_ids torch.Size([16, 100])


In [17]:
import torch
import torch.nn as nn
from transformers import AutoModel

class RefindRelationClassifier(nn.Module):
    def __init__(self, encoder_name, num_labels, dropout=0.1):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(encoder_name)
        hidden_size = self.encoder.config.hidden_size

        # [CLS] + subj + obj  ->  3 * hidden_size
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size * 3, hidden_size),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_size, num_labels)
        )

    def forward(self, input_ids, attention_mask, subj_mask, obj_mask, token_type_ids=None, labels=None):
        encoder_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        sequence_output = encoder_outputs.last_hidden_state  # [B, L, H]

        # [CLS] embedding at position 0
        cls_repr = sequence_output[:, 0, :]  # [B, H]

        # Expand masks to [B, L, 1]
        subj_mask = subj_mask.unsqueeze(-1)  # [B, L, 1]
        obj_mask  = obj_mask.unsqueeze(-1)   # [B, L, 1]

        # Avoid division by zero by clamping denominator
        subj_sum = (sequence_output * subj_mask).sum(dim=1)                # [B, H]
        subj_len = subj_mask.sum(dim=1).clamp(min=1e-6)                    # [B, 1]
        subj_repr = subj_sum / subj_len                                   # [B, H]

        obj_sum = (sequence_output * obj_mask).sum(dim=1)
        obj_len = obj_mask.sum(dim=1).clamp(min=1e-6)
        obj_repr = obj_sum / obj_len

        pair_repr = torch.cat([cls_repr, subj_repr, obj_repr], dim=-1)    # [B, 3H]

        logits = self.classifier(pair_repr)                                # [B, num_labels]

        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits, labels)

        return {"loss": loss, "logits": logits}


In [37]:
!pip install --upgrade ml_dtypes



  Using cached ml_dtypes-0.5.3-cp311-cp311-win_amd64.whl.metadata (9.2 kB)
Using cached ml_dtypes-0.5.3-cp311-cp311-win_amd64.whl (206 kB)
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
tensorflow-intel 2.15.0 requires ml-dtypes~=0.2.0, but you have ml-dtypes 0.5.3 which is incompatible.


In [39]:
!pip uninstall -y jax jaxlib tensorflow tensorflow-intel


Found existing installation: jax 0.7.2
Uninstalling jax-0.7.2:
  Successfully uninstalled jax-0.7.2
Found existing installation: jaxlib 0.7.2
Uninstalling jaxlib-0.7.2:
  Successfully uninstalled jaxlib-0.7.2
Found existing installation: tensorflow 2.15.0
Uninstalling tensorflow-2.15.0:
  Successfully uninstalled tensorflow-2.15.0
Found existing installation: tensorflow-intel 2.15.0
Uninstalling tensorflow-intel-2.15.0:
  Successfully uninstalled tensorflow-intel-2.15.0


You can safely remove it manually.
You can safely remove it manually.


In [19]:
import math
from tqdm.auto import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

num_labels = len(rel2id)
model = RefindRelationClassifier(MODEL_NAME, num_labels=num_labels).to(device)

EPOCHS = 3
LR = 2e-5
optimizer = torch.optim.AdamW(model.parameters(), lr=LR)


Using device: cuda


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

C:\Users\shaha\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaha\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [21]:
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

def evaluate(model, data_loader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                subj_mask=batch["subj_mask"],
                obj_mask=batch["obj_mask"],
                token_type_ids=batch.get("token_type_ids", None),
            )
            logits = outputs["logits"]
            preds = logits.argmax(dim=-1)

            all_labels.extend(batch["labels"].cpu().tolist())
            all_preds.extend(preds.cpu().tolist())

    prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="micro", zero_division=0)
    acc = accuracy_score(all_labels, all_preds)
    return acc, prec, rec, f1


In [31]:
best_dev_f1 = 0.0
best_state_dict = None

for epoch in range(1, EPOCHS + 1):
    model.train()
    running_loss = 0.0
    steps = 0

    for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
        batch = {k: v.to(device) for k, v in batch.items()}

        outputs = model(
            input_ids=batch["input_ids"],
            attention_mask=batch["attention_mask"],
            subj_mask=batch["subj_mask"],
            obj_mask=batch["obj_mask"],
            token_type_ids=batch.get("token_type_ids", None),
            labels=batch["labels"],
        )
        loss = outputs["loss"]

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        optimizer.zero_grad()

        running_loss += loss.item()
        steps += 1

    avg_loss = running_loss / max(steps, 1)
    dev_acc, dev_prec, dev_rec, dev_f1 = evaluate(model, dev_loader)
    print(f"\nEpoch {epoch} - train loss: {avg_loss:.4f}")
    print(f"Dev  -> acc: {dev_acc:.4f}, prec: {dev_prec:.4f}, rec: {dev_rec:.4f}, F1: {dev_f1:.4f}")

    if dev_f1 > best_dev_f1:
        best_dev_f1 = dev_f1
        best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}
        print(f"  🟢 New best model saved with dev F1 = {best_dev_f1:.4f}")

# Load best model back to device
if best_state_dict is not None:
    model.load_state_dict(best_state_dict)
    model.to(device)


Epoch 1:   0%|          | 0/1255 [00:00<?, ?it/s]


Epoch 1 - train loss: 0.5945
Dev  -> acc: 0.8460, prec: 0.8460, rec: 0.8460, F1: 0.8460
  🟢 New best model saved with dev F1 = 0.8460


Epoch 2:   0%|          | 0/1255 [00:00<?, ?it/s]


Epoch 2 - train loss: 0.2907
Dev  -> acc: 0.8484, prec: 0.8484, rec: 0.8484, F1: 0.8484
  🟢 New best model saved with dev F1 = 0.8484


Epoch 3:   0%|          | 0/1255 [00:00<?, ?it/s]


Epoch 3 - train loss: 0.2074
Dev  -> acc: 0.8430, prec: 0.8430, rec: 0.8430, F1: 0.8430


In [33]:
test_acc, test_prec, test_rec, test_f1 = evaluate(model, test_loader)
print(f"TEST -> acc: {test_acc:.4f}, prec: {test_prec:.4f}, rec: {test_rec:.4f}, F1: {test_f1:.4f}")


TEST -> acc: 0.7709, prec: 0.7709, rec: 0.7709, F1: 0.7709


In [5]:
def run_refind_experiment(
    model_name,
    max_len=256,
    batch_size=16,
    epochs=3,
    lr=2e-5,
    device=None,
):
    from transformers import AutoTokenizer, AutoModel
    import torch
    import torch.nn as nn
    from torch.utils.data import DataLoader
    from tqdm.notebook import tqdm
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score

    print(f"=== Running experiment with encoder: {model_name} ===")

    # 1) tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 2) dataset + dataloaders (reuse normalize_example outputs: train_proc/dev_proc/test_proc)
    class RefindDataset(Dataset):
        def __init__(self, examples, tokenizer, max_len=256):
            self.examples = examples
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            ex = self.examples[idx]
            tokens = ex["tokens"]
            e1 = ex["e1"]
            e2 = ex["e2"]
            label_id = ex["rel_id"]

            encoding = self.tokenizer(
                tokens,
                is_split_into_words=True,
                truncation=True,
                max_length=self.max_len,
                return_attention_mask=True,
                return_token_type_ids=True,
            )

            word_ids = encoding.word_ids()

            subj_mask = [0] * len(word_ids)
            obj_mask  = [0] * len(word_ids)

            for i, w_id in enumerate(word_ids):
                if w_id is None:
                    continue
                if e1["start"] <= w_id < e1["end"]:
                    subj_mask[i] = 1
                if e2["start"] <= w_id < e2["end"]:
                    obj_mask[i] = 1

            item = {
                "input_ids": torch.tensor(encoding["input_ids"], dtype=torch.long),
                "attention_mask": torch.tensor(encoding["attention_mask"], dtype=torch.long),
                "subj_mask": torch.tensor(subj_mask, dtype=torch.float),
                "obj_mask": torch.tensor(obj_mask, dtype=torch.float),
                "labels": torch.tensor(label_id, dtype=torch.long),
            }
            if "token_type_ids" in encoding:
                item["token_type_ids"] = torch.tensor(encoding["token_type_ids"], dtype=torch.long)

            return item

    def collate_fn(batch):
        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
        max_len_local = max(len(ex["input_ids"]) for ex in batch)

        def pad_seq(seq, pad_value):
            seq = seq.tolist() if isinstance(seq, torch.Tensor) else list(seq)
            return seq + [pad_value] * (max_len_local - len(seq))

        input_ids = torch.tensor(
            [pad_seq(ex["input_ids"], pad_token_id) for ex in batch],
            dtype=torch.long,
        )
        attention_mask = torch.tensor(
            [pad_seq(ex["attention_mask"], 0) for ex in batch],
            dtype=torch.long,
        )
        subj_mask = torch.tensor(
            [pad_seq(ex["subj_mask"], 0.0) for ex in batch],
            dtype=torch.float,
        )
        obj_mask = torch.tensor(
            [pad_seq(ex["obj_mask"], 0.0) for ex in batch],
            dtype=torch.float,
        )
        labels = torch.tensor(
            [int(ex["labels"]) for ex in batch],
            dtype=torch.long,
        )

        batch_dict = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "subj_mask": subj_mask,
            "obj_mask": obj_mask,
            "labels": labels,
        }

        if "token_type_ids" in batch[0]:
            token_type_ids = torch.tensor(
                [pad_seq(ex["token_type_ids"], 0) for ex in batch],
                dtype=torch.long,
            )
            batch_dict["token_type_ids"] = token_type_ids

        return batch_dict

    train_ds = RefindDataset(train_proc, tokenizer, max_len=max_len)
    dev_ds   = RefindDataset(dev_proc,   tokenizer, max_len=max_len)
    test_ds  = RefindDataset(test_proc,  tokenizer, max_len=max_len)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  collate_fn=collate_fn)
    dev_loader   = DataLoader(dev_ds,   batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # 3) model
    class RefindRelationClassifier(nn.Module):
        def __init__(self, encoder_name, num_labels, dropout=0.1):
            super().__init__()
            self.encoder = AutoModel.from_pretrained(encoder_name)
            hidden_size = self.encoder.config.hidden_size
            self.classifier = nn.Sequential(
                nn.Linear(hidden_size * 3, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, num_labels)
            )

        def forward(self, input_ids, attention_mask, subj_mask, obj_mask, token_type_ids=None, labels=None):
            encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
            sequence_output = encoder_outputs.last_hidden_state
            cls_repr = sequence_output[:, 0, :]

            subj_mask_ = subj_mask.unsqueeze(-1)
            obj_mask_  = obj_mask.unsqueeze(-1)

            subj_sum = (sequence_output * subj_mask_).sum(dim=1)
            subj_len = subj_mask_.sum(dim=1).clamp(min=1e-6)
            subj_repr = subj_sum / subj_len

            obj_sum = (sequence_output * obj_mask_).sum(dim=1)
            obj_len = obj_mask_.sum(dim=1).clamp(min=1e-6)
            obj_repr = obj_sum / obj_len

            pair_repr = torch.cat([cls_repr, subj_repr, obj_repr], dim=-1)
            logits = self.classifier(pair_repr)

            loss = None
            if labels is not None:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits, labels)

            return {"loss": loss, "logits": logits}

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    num_labels = len(rel2id)
    model = RefindRelationClassifier(model_name, num_labels=num_labels).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    def evaluate(model, data_loader):
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for batch in data_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    subj_mask=batch["subj_mask"],
                    obj_mask=batch["obj_mask"],
                    token_type_ids=batch.get("token_type_ids", None),
                )
                logits = outputs["logits"]
                preds = logits.argmax(dim=-1)
                all_labels.extend(batch["labels"].cpu().tolist())
                all_preds.extend(preds.cpu().tolist())

        prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="micro", zero_division=0)
        acc = accuracy_score(all_labels, all_preds)
        return acc, prec, rec, f1

    best_dev_f1 = 0.0
    best_state_dict = None

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        steps = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                subj_mask=batch["subj_mask"],
                obj_mask=batch["obj_mask"],
                token_type_ids=batch.get("token_type_ids", None),
                labels=batch["labels"],
            )
            loss = outputs["loss"]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()
            running_loss += loss.item()
            steps += 1

        avg_loss = running_loss / max(steps, 1)
        dev_acc, dev_prec, dev_rec, dev_f1 = evaluate(model, dev_loader)
        print(f"\nEpoch {epoch} - train loss: {avg_loss:.4f}")
        print(f"Dev -> acc: {dev_acc:.4f}, prec: {dev_prec:.4f}, rec: {dev_rec:.4f}, F1: {dev_f1:.4f}")

        if dev_f1 > best_dev_f1:
            best_dev_f1 = dev_f1
            best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}
            print(f"  🟢 New best model saved with dev F1 = {best_dev_f1:.4f}")

    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)
        model.to(device)

    test_acc, test_prec, test_rec, test_f1 = evaluate(model, test_loader)
    print(f"\nTEST -> acc: {test_acc:.4f}, prec: {test_prec:.4f}, rec: {test_rec:.4f}, F1: {test_f1:.4f}")

    return {
        "dev_f1": best_dev_f1,
        "test_acc": test_acc,
        "test_prec": test_prec,
        "test_rec": test_rec,
        "test_f1": test_f1,
    }


In [40]:
!pip install --upgrade "torch>=2.6.0"


   ---------------------------------------- 0.0/111.0 MB ? eta -:--:--
   ---------------------------------------- 0.1/111.0 MB 3.3 MB/s eta 0:00:34
   ---------------------------------------- 0.4/111.0 MB 5.3 MB/s eta 0:00:22
   ---------------------------------------- 0.9/111.0 MB 6.8 MB/s eta 0:00:17
   ---------------------------------------- 1.4/111.0 MB 7.8 MB/s eta 0:00:15
    --------------------------------------- 1.9/111.0 MB 8.7 MB/s eta 0:00:13
    --------------------------------------- 2.5/111.0 MB 9.5 MB/s eta 0:00:12
   - -------------------------------------- 3.1/111.0 MB 9.8 MB/s eta 0:00:11
   - -------------------------------------- 3.7/111.0 MB 10.3 MB/s eta 0:00:11
   - -------------------------------------- 4.4/111.0 MB 10.8 MB/s eta 0:00:10
   - -------------------------------------- 4.9/111.0 MB 10.9 MB/s eta 0:00:10
   -- ------------------------------------- 5.6/111.0 MB 11.1 MB/s eta 0:00:10
   -- ------------------------------------- 6.2/111.0 MB 11.3 MB/s 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1 requires torch==2.5.1, but you have torch 2.9.1 which is incompatible.
torchvision 0.20.1 requires torch==2.5.1, but you have torch 2.9.1 which is incompatible.


In [1]:
import os, json
from collections import Counter
from itertools import chain

ROOT = "ee782_re_project"
refind_pub = os.path.join(ROOT, "data", "REFinD", "public_dat")

def load_refind_split(split_name):
    fname = f"{split_name}_refind_official.json"
    path = os.path.join(refind_pub, fname)
    print(f"Loading {split_name} from {path}")
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"  -> {len(data)} examples")
    return data

train_raw = load_refind_split("train")
dev_raw   = load_refind_split("dev")
test_raw  = load_refind_split("test")

all_rels = sorted({ex["relation"] for ex in chain(train_raw, dev_raw, test_raw)})
rel2id = {rel: i for i, rel in enumerate(all_rels)}
id2rel = {i: rel for rel, i in rel2id.items()}

def normalize_example(ex):
    tokens = ex["token"]
    e1 = {"start": ex["e1_start"], "end": ex["e1_end"], "type": ex["e1_type"]}
    e2 = {"start": ex["e2_start"], "end": ex["e2_end"], "type": ex["e2_type"]}
    rel_label = ex["relation"]
    return {
        "docid": ex["docid"],
        "tokens": tokens,
        "e1": e1,
        "e2": e2,
        "rel": rel_label,
        "rel_id": rel2id[rel_label],
        "rel_group": ex["rel_group"],
    }

train_proc = [normalize_example(ex) for ex in train_raw]
dev_proc   = [normalize_example(ex) for ex in dev_raw]
test_proc  = [normalize_example(ex) for ex in test_raw]

print("Train/dev/test sizes:", len(train_proc), len(dev_proc), len(test_proc))
print(train_proc[0])


Loading train from ee782_re_project\data\REFinD\public_dat\train_refind_official.json
  -> 20070 examples
Loading dev from ee782_re_project\data\REFinD\public_dat\dev_refind_official.json
  -> 4306 examples
Loading test from ee782_re_project\data\REFinD\public_dat\test_refind_official.json
  -> 4300 examples
Train/dev/test sizes: 20070 4306 4300
{'docid': '2016/2017', 'tokens': ['warrants', 'to', 'purchase', 'Lumos', 'Networks', 'Corp.', 'common', 'stock', ',', 'the', 'Pamplona', 'Entities', ')', '.'], 'e1': {'start': 3, 'end': 6, 'type': 'ORG'}, 'e2': {'start': 9, 'end': 12, 'type': 'ORG'}, 'rel': 'no_relation', 'rel_id': 0, 'rel_group': 'ORG-ORG'}


In [3]:
def run_refind_experiment(
    model_name,
    train_examples,
    dev_examples,
    test_examples,
    rel2id,
    max_len=256,
    batch_size=16,
    epochs=3,
    lr=2e-5,
    device=None,
):
    """
    model_name: HF ID or local path (e.g. "bert-base-uncased" or "SALT-NLP/FLANG-BERT")
    train_examples/dev_examples/test_examples: lists of normalized dicts (train_proc/dev_proc/test_proc)
    rel2id: relation -> id mapping
    """
    import torch
    import torch.nn as nn
    from torch.utils.data import Dataset, DataLoader
    from transformers import AutoTokenizer, AutoModel
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    from tqdm.auto import tqdm  # works in most Jupyter setups

    print(f"\n=== Running experiment with encoder: {model_name} ===")

    # 1) Tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 2) Dataset + DataLoader
    class RefindDataset(Dataset):
        def __init__(self, examples, tokenizer, max_len=256):
            self.examples = examples
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            ex = self.examples[idx]
            tokens = ex["tokens"]
            e1 = ex["e1"]
            e2 = ex["e2"]
            label_id = ex["rel_id"]

            encoding = self.tokenizer(
                tokens,
                is_split_into_words=True,
                truncation=True,
                max_length=self.max_len,
                return_attention_mask=True,
                return_token_type_ids=True,
            )

            word_ids = encoding.word_ids()

            subj_mask = [0] * len(word_ids)
            obj_mask  = [0] * len(word_ids)

            for i, w_id in enumerate(word_ids):
                if w_id is None:
                    continue
                if e1["start"] <= w_id < e1["end"]:
                    subj_mask[i] = 1
                if e2["start"] <= w_id < e2["end"]:
                    obj_mask[i] = 1

            item = {
                "input_ids": torch.tensor(encoding["input_ids"], dtype=torch.long),
                "attention_mask": torch.tensor(encoding["attention_mask"], dtype=torch.long),
                "subj_mask": torch.tensor(subj_mask, dtype=torch.float),
                "obj_mask": torch.tensor(obj_mask, dtype=torch.float),
                "labels": torch.tensor(label_id, dtype=torch.long),
            }
            if "token_type_ids" in encoding:
                item["token_type_ids"] = torch.tensor(encoding["token_type_ids"], dtype=torch.long)

            return item

    def collate_fn(batch):
        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
        max_len_local = max(len(ex["input_ids"]) for ex in batch)

        def pad_seq(seq, pad_value):
            seq = seq.tolist() if isinstance(seq, torch.Tensor) else list(seq)
            return seq + [pad_value] * (max_len_local - len(seq))

        input_ids = torch.tensor(
            [pad_seq(ex["input_ids"], pad_token_id) for ex in batch],
            dtype=torch.long,
        )
        attention_mask = torch.tensor(
            [pad_seq(ex["attention_mask"], 0) for ex in batch],
            dtype=torch.long,
        )
        subj_mask = torch.tensor(
            [pad_seq(ex["subj_mask"], 0.0) for ex in batch],
            dtype=torch.float,
        )
        obj_mask = torch.tensor(
            [pad_seq(ex["obj_mask"], 0.0) for ex in batch],
            dtype=torch.float,
        )
        labels = torch.tensor(
            [int(ex["labels"]) for ex in batch],
            dtype=torch.long,
        )

        batch_dict = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "subj_mask": subj_mask,
            "obj_mask": obj_mask,
            "labels": labels,
        }

        if "token_type_ids" in batch[0]:
            token_type_ids = torch.tensor(
                [pad_seq(ex["token_type_ids"], 0) for ex in batch],
                dtype=torch.long,
            )
            batch_dict["token_type_ids"] = token_type_ids

        return batch_dict

    train_ds = RefindDataset(train_examples, tokenizer, max_len=max_len)
    dev_ds   = RefindDataset(dev_examples,   tokenizer, max_len=max_len)
    test_ds  = RefindDataset(test_examples,  tokenizer, max_len=max_len)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  collate_fn=collate_fn)
    dev_loader   = DataLoader(dev_ds,   batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    # 3) Model
    class RefindRelationClassifier(nn.Module):
        def __init__(self, encoder_name, num_labels, dropout=0.1):
            super().__init__()
            self.encoder = AutoModel.from_pretrained(encoder_name)
            hidden_size = self.encoder.config.hidden_size
            self.classifier = nn.Sequential(
                nn.Linear(hidden_size * 3, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, num_labels)
            )

        def forward(self, input_ids, attention_mask, subj_mask, obj_mask, token_type_ids=None, labels=None):
            encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )
            sequence_output = encoder_outputs.last_hidden_state  # [B, L, H]

            cls_repr = sequence_output[:, 0, :]  # [B, H]

            subj_mask_ = subj_mask.unsqueeze(-1)
            obj_mask_  = obj_mask.unsqueeze(-1)

            subj_sum = (sequence_output * subj_mask_).sum(dim=1)
            subj_len = subj_mask_.sum(dim=1).clamp(min=1e-6)
            subj_repr = subj_sum / subj_len

            obj_sum = (sequence_output * obj_mask_).sum(dim=1)
            obj_len = obj_mask_.sum(dim=1).clamp(min=1e-6)
            obj_repr = obj_sum / obj_len

            pair_repr = torch.cat([cls_repr, subj_repr, obj_repr], dim=-1)
            logits = self.classifier(pair_repr)

            loss = None
            if labels is not None:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits, labels)

            return {"loss": loss, "logits": logits}

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    num_labels = len(rel2id)
    model = RefindRelationClassifier(model_name, num_labels=num_labels).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    def evaluate(model, data_loader):
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for batch in data_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    subj_mask=batch["subj_mask"],
                    obj_mask=batch["obj_mask"],
                    token_type_ids=batch.get("token_type_ids", None),
                )
                logits = outputs["logits"]
                preds = logits.argmax(dim=-1)
                all_labels.extend(batch["labels"].cpu().tolist())
                all_preds.extend(preds.cpu().tolist())

        prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="micro", zero_division=0)
        acc = accuracy_score(all_labels, all_preds)
        return acc, prec, rec, f1

    # 4) Training loop
    best_dev_f1 = 0.0
    best_state_dict = None

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        steps = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                subj_mask=batch["subj_mask"],
                obj_mask=batch["obj_mask"],
                token_type_ids=batch.get("token_type_ids", None),
                labels=batch["labels"],
            )
            loss = outputs["loss"]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()
            running_loss += loss.item()
            steps += 1

        avg_loss = running_loss / max(steps, 1)
        dev_acc, dev_prec, dev_rec, dev_f1 = evaluate(model, dev_loader)
        print(f"\nEpoch {epoch} - train loss: {avg_loss:.4f}")
        print(f"Dev -> acc: {dev_acc:.4f}, prec: {dev_prec:.4f}, rec: {dev_rec:.4f}, F1: {dev_f1:.4f}")

        if dev_f1 > best_dev_f1:
            best_dev_f1 = dev_f1
            best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}
            print(f"  🟢 New best model saved with dev F1 = {best_dev_f1:.4f}")

    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)
        model.to(device)

    test_acc, test_prec, test_rec, test_f1 = evaluate(model, test_loader)
    print(f"\nTEST -> acc: {test_acc:.4f}, prec: {test_prec:.4f}, rec: {test_rec:.4f}, F1: {test_f1:.4f}")

    return {
        "dev_f1": best_dev_f1,
        "test_acc": test_acc,
        "test_prec": test_prec,
        "test_rec": test_rec,
        "test_f1": test_f1,
    }


In [2]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))



Torch version: 2.5.1
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [4]:
import os, json
from itertools import chain

ROOT = "ee782_re_project"
refind_pub = os.path.join(ROOT, "data", "REFinD", "public_dat")

def load_refind_split(split_name):
    fname = f"{split_name}_refind_official.json"
    path = os.path.join(refind_pub, fname)
    print(f"Loading {split_name} from {path}")
    with open(path, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"  -> {len(data)} examples")
    return data

train_raw = load_refind_split("train")
dev_raw   = load_refind_split("dev")
test_raw  = load_refind_split("test")

all_rels = sorted({ex["relation"] for ex in chain(train_raw, dev_raw, test_raw)})
rel2id = {rel: i for i, rel in enumerate(all_rels)}
id2rel = {i: rel for rel, i in rel2id.items()}

def normalize_example(ex):
    tokens = ex["token"]
    e1 = {"start": ex["e1_start"], "end": ex["e1_end"], "type": ex["e1_type"]}
    e2 = {"start": ex["e2_start"], "end": ex["e2_end"], "type": ex["e2_type"]}
    rel_label = ex["relation"]
    return {
        "docid": ex["docid"],
        "tokens": tokens,
        "e1": e1,
        "e2": e2,
        "rel": rel_label,
        "rel_id": rel2id[rel_label],
        "rel_group": ex["rel_group"],
    }

train_proc = [normalize_example(ex) for ex in train_raw]
dev_proc   = [normalize_example(ex) for ex in dev_raw]
test_proc  = [normalize_example(ex) for ex in test_raw]

print("Train/dev/test sizes:", len(train_proc), len(dev_proc), len(test_proc))
print(train_proc[0])


Loading train from ee782_re_project\data\REFinD\public_dat\train_refind_official.json
  -> 20070 examples
Loading dev from ee782_re_project\data\REFinD\public_dat\dev_refind_official.json
  -> 4306 examples
Loading test from ee782_re_project\data\REFinD\public_dat\test_refind_official.json
  -> 4300 examples
Train/dev/test sizes: 20070 4306 4300
{'docid': '2016/2017', 'tokens': ['warrants', 'to', 'purchase', 'Lumos', 'Networks', 'Corp.', 'common', 'stock', ',', 'the', 'Pamplona', 'Entities', ')', '.'], 'e1': {'start': 3, 'end': 6, 'type': 'ORG'}, 'e2': {'start': 9, 'end': 12, 'type': 'ORG'}, 'rel': 'no_relation', 'rel_id': 0, 'rel_group': 'ORG-ORG'}


In [6]:
def run_refind_experiment(
    model_name,
    train_examples,
    dev_examples,
    test_examples,
    rel2id,
    max_len=256,
    batch_size=16,
    epochs=3,
    lr=2e-5,
    device=None,
):
    import torch
    import torch.nn as nn
    from torch.utils.data import Dataset, DataLoader
    from transformers import AutoTokenizer, AutoModel
    from sklearn.metrics import precision_recall_fscore_support, accuracy_score
    from tqdm.auto import tqdm

    print(f"\n=== Running experiment with encoder: {model_name} ===")

    tokenizer = AutoTokenizer.from_pretrained(model_name)

    class RefindDataset(Dataset):
        def __init__(self, examples, tokenizer, max_len=256):
            self.examples = examples
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.examples)

        def __getitem__(self, idx):
            ex = self.examples[idx]
            tokens = ex["tokens"]
            e1 = ex["e1"]
            e2 = ex["e2"]
            label_id = ex["rel_id"]

            encoding = self.tokenizer(
                tokens,
                is_split_into_words=True,
                truncation=True,
                max_length=self.max_len,
                return_attention_mask=True,
                return_token_type_ids=True,
            )

            word_ids = encoding.word_ids()

            subj_mask = [0] * len(word_ids)
            obj_mask  = [0] * len(word_ids)

            for i, w_id in enumerate(word_ids):
                if w_id is None:
                    continue
                if e1["start"] <= w_id < e1["end"]:
                    subj_mask[i] = 1
                if e2["start"] <= w_id < e2["end"]:
                    obj_mask[i] = 1

            item = {
                "input_ids": torch.tensor(encoding["input_ids"], dtype=torch.long),
                "attention_mask": torch.tensor(encoding["attention_mask"], dtype=torch.long),
                "subj_mask": torch.tensor(subj_mask, dtype=torch.float),
                "obj_mask": torch.tensor(obj_mask, dtype=torch.float),
                "labels": torch.tensor(label_id, dtype=torch.long),
            }
            if "token_type_ids" in encoding:
                item["token_type_ids"] = torch.tensor(encoding["token_type_ids"], dtype=torch.long)

            return item

    def collate_fn(batch):
        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
        max_len_local = max(len(ex["input_ids"]) for ex in batch)

        def pad_seq(seq, pad_value):
            seq = seq.tolist() if isinstance(seq, torch.Tensor) else list(seq)
            return seq + [pad_value] * (max_len_local - len(seq))

        input_ids = torch.tensor(
            [pad_seq(ex["input_ids"], pad_token_id) for ex in batch],
            dtype=torch.long,
        )
        attention_mask = torch.tensor(
            [pad_seq(ex["attention_mask"], 0) for ex in batch],
            dtype=torch.long,
        )
        subj_mask = torch.tensor(
            [pad_seq(ex["subj_mask"], 0.0) for ex in batch],
            dtype=torch.float,
        )
        obj_mask = torch.tensor(
            [pad_seq(ex["obj_mask"], 0.0) for ex in batch],
            dtype=torch.float,
        )
        labels = torch.tensor(
            [int(ex["labels"]) for ex in batch],
            dtype=torch.long,
        )

        batch_dict = {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "subj_mask": subj_mask,
            "obj_mask": obj_mask,
            "labels": labels,
        }

        if "token_type_ids" in batch[0]:
            token_type_ids = torch.tensor(
                [pad_seq(ex["token_type_ids"], 0) for ex in batch],
                dtype=torch.long,
            )
            batch_dict["token_type_ids"] = token_type_ids

        return batch_dict

    train_ds = RefindDataset(train_examples, tokenizer, max_len=max_len)
    dev_ds   = RefindDataset(dev_examples,   tokenizer, max_len=max_len)
    test_ds  = RefindDataset(test_examples,  tokenizer, max_len=max_len)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  collate_fn=collate_fn)
    dev_loader   = DataLoader(dev_ds,   batch_size=batch_size, shuffle=False, collate_fn=collate_fn)
    test_loader  = DataLoader(test_ds,  batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

    class RefindRelationClassifier(nn.Module):
        def __init__(self, encoder_name, num_labels, dropout=0.1):
            super().__init__()
            self.encoder = AutoModel.from_pretrained(encoder_name)
            hidden_size = self.encoder.config.hidden_size
            self.classifier = nn.Sequential(
                nn.Linear(hidden_size * 3, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_size, num_labels)
            )

        def forward(self, input_ids, attention_mask, subj_mask, obj_mask, token_type_ids=None, labels=None):
            encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
            )
            sequence_output = encoder_outputs.last_hidden_state  # [B, L, H]

            cls_repr = sequence_output[:, 0, :]  # [B, H]

            subj_mask_ = subj_mask.unsqueeze(-1)
            obj_mask_  = obj_mask.unsqueeze(-1)

            subj_sum = (sequence_output * subj_mask_).sum(dim=1)
            subj_len = subj_mask_.sum(dim=1).clamp(min=1e-6)
            subj_repr = subj_sum / subj_len

            obj_sum = (sequence_output * obj_mask_).sum(dim=1)
            obj_len = obj_mask_.sum(dim=1).clamp(min=1e-6)
            obj_repr = obj_sum / obj_len

            pair_repr = torch.cat([cls_repr, subj_repr, obj_repr], dim=-1)
            logits = self.classifier(pair_repr)

            loss = None
            if labels is not None:
                loss_fct = nn.CrossEntropyLoss()
                loss = loss_fct(logits, labels)

            return {"loss": loss, "logits": logits}

    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using device:", device)

    num_labels = len(rel2id)
    model = RefindRelationClassifier(model_name, num_labels=num_labels).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    def evaluate(model, data_loader):
        model.eval()
        all_labels = []
        all_preds = []
        with torch.no_grad():
            for batch in data_loader:
                batch = {k: v.to(device) for k, v in batch.items()}
                outputs = model(
                    input_ids=batch["input_ids"],
                    attention_mask=batch["attention_mask"],
                    subj_mask=batch["subj_mask"],
                    obj_mask=batch["obj_mask"],
                    token_type_ids=batch.get("token_type_ids", None),
                )
                logits = outputs["logits"]
                preds = logits.argmax(dim=-1)
                all_labels.extend(batch["labels"].cpu().tolist())
                all_preds.extend(preds.cpu().tolist())

        prec, rec, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average="micro", zero_division=0)
        acc = accuracy_score(all_labels, all_preds)
        return acc, prec, rec, f1

    best_dev_f1 = 0.0
    best_state_dict = None

    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        steps = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch}"):
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(
                input_ids=batch["input_ids"],
                attention_mask=batch["attention_mask"],
                subj_mask=batch["subj_mask"],
                obj_mask=batch["obj_mask"],
                token_type_ids=batch.get("token_type_ids", None),
                labels=batch["labels"],
            )
            loss = outputs["loss"]
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()
            running_loss += loss.item()
            steps += 1

        avg_loss = running_loss / max(steps, 1)
        dev_acc, dev_prec, dev_rec, dev_f1 = evaluate(model, dev_loader)
        print(f"\nEpoch {epoch} - train loss: {avg_loss:.4f}")
        print(f"Dev -> acc: {dev_acc:.4f}, prec: {dev_prec:.4f}, rec: {dev_rec:.4f}, F1: {dev_f1:.4f}")

        if dev_f1 > best_dev_f1:
            best_dev_f1 = dev_f1
            best_state_dict = {k: v.cpu() for k, v in model.state_dict().items()}
            print(f"  🟢 New best model saved with dev F1 = {best_dev_f1:.4f}")

    if best_state_dict is not None:
        model.load_state_dict(best_state_dict)
        model.to(device)

    test_acc, test_prec, test_rec, test_f1 = evaluate(model, test_loader)
    print(f"\nTEST -> acc: {test_acc:.4f}, prec: {test_prec:.4f}, rec: {test_rec:.4f}, F1: {test_f1:.4f}")

    return {
        "dev_f1": best_dev_f1,
        "test_acc": test_acc,
        "test_prec": test_prec,
        "test_rec": test_rec,
        "test_f1": test_f1,
    }


In [8]:
results_bert_quick = run_refind_experiment(
    "bert-base-uncased",
    train_proc, dev_proc, test_proc, rel2id,
    epochs=1, batch_size=16, max_len=256,
)


C:\Users\shaha\anaconda3\envs\ee782gpu\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



=== Running experiment with encoder: bert-base-uncased ===
Using device: cuda


Epoch 1: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 1255/1255 [05:31<00:00,  3.79it/s]



Epoch 1 - train loss: 0.5877
Dev -> acc: 0.8423, prec: 0.8423, rec: 0.8423, F1: 0.8423
  🟢 New best model saved with dev F1 = 0.8423

TEST -> acc: 0.7321, prec: 0.7321, rec: 0.7321, F1: 0.7321


In [12]:
!pip install --upgrade "torch>=2.6.0"


  Using cached torch-2.9.1-cp311-cp311-win_amd64.whl.metadata (30 kB)
Using cached torch-2.9.1-cp311-cp311-win_amd64.whl (111.0 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1
    Uninstalling torch-2.5.1:
      Successfully uninstalled torch-2.5.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1 requires torch==2.5.1, but you have torch 2.9.1 which is incompatible.
torchvision 0.20.1 requires torch==2.5.1, but you have torch 2.9.1 which is incompatible.


In [2]:
import torch
import torchvision

print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))

print("Torchvision version:", torchvision.__version__)


Torch version: 2.9.1+cu126
CUDA available: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU
Torchvision version: 0.24.1+cu126
